## TO DO
add accuracy plots, confusion matrix

kfolds, stratified kfold

scores by # words used for classification and dataset

track best words for classification

PCA?

add phase for angle of arrival as well, not just amplitude attenuation

In [20]:
#from imblearn.over_sampling import RandomOverSampler, SMOTE
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA

import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn import svm 
from sklearn.metrics import accuracy_score
from statistics import mean
import random
from scipy.io import loadmat

In [21]:
# Variables to tweak
num_words = 20
dset = 'dl'
#kfolds = 

In [22]:
d = 'datasets/'

if dset == 'dl':
    x = loadmat(d + 'dataset_lab_276_dl.mat')
    data = x['csid_lab']
    print(x['csid_lab'][0].shape)
    
elif dset == 'ul':
    x = loadmat(d + 'dataset_lab_276_ul.mat')
    data = x['csiu_lab']
    print(x['csiu_lab'][0].shape)
    
elif dset == 'home':
    x = loadmat(d + 'dataset_home_276.mat')
    data = x['csid_home']
    print(x['csid_home'][0].shape)
    
print(x.keys())

(30, 3, 5520)
['csid_lab', '__version__', 'label_lab', '__header__', '__globals__']


In [23]:
with open('sign_labels.csv') as f:
    labels = f.readlines()[0].split('\r')
    labels[0] = labels[0][3:]               # whitespace error with first element
    print(len(labels))

# make ground truth for each element
label_ind = [i[0]-1 for i in x['label_lab']]
print(len(label_ind))
print(labels[label_ind[0]]) #first label in dataset

276
5520
FINISH


In [24]:
random_words = []
while(len(random_words)!=num_words):
    a = int(random.random() * 276)
    if a not in random_words:
        random_words.append(a)

In [25]:
label_indices = []

for i in random_words:
    label_indices.append(np.where(x['label_lab']==[i])[0]) # indices of where labels are the same 
                                                           # throughout dataset (same words)
# words being used
for i in range(0, len(label_indices)):
    print(labels[label_indices[i][0]])

COLOR
CUTE
WHO
BOOK
GROSS
STUDENT
YOUNG
SO-SO
WITH
MATH
GO
AGAIN
FAMILY
NOT
ORANGE
CLEAN
MEAN
AWFUL
TV
WHERE


In [26]:
# randomly sample percent% from each sample for testing

def get_rand_indices(size, percent):
    test_set_size = int(size * percent)
    test_set_ind = []

    while(len(test_set_ind)<=test_set_size):
        rand = int(random.random() * size)
        if rand not in test_set_ind:
            test_set_ind.append(rand)
    return test_set_ind


get_rand_indices(20, .2)

[2, 11, 15, 0, 5]

In [27]:
x_train = []
x_test = []
y_train = []
y_test = []

for i in label_indices:
    test_indices = get_rand_indices(len(i), .2)
    train_indices = [j for j in range(0, len(i)) if j not in test_indices]
    for j in test_indices:
        x_test.append(data[:,:,:,i[j]])
        y_test.append(i[j]%276)
        
    for j in train_indices:
        x_train.append(data[:,:,:,i[j]])
        y_train.append(i[j]%276)

In [28]:
print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

(300, 200, 30, 3)
(300,)
(100, 200, 30, 3)
(100,)


In [29]:
# simplify indexes to [0 - num_words]

def to_categorical_(train, test):
    cur = 0
    words = dict.fromkeys(test)
    for i in words.keys():
        words[i] = cur
        cur += 1
    return ([words[i] for i in train], [words[i] for i in test])

y_train, y_test = to_categorical_(y_train, y_test)

In [30]:
# map(abs, x) -> amplitide of complex #'s, model is currently looking at amplitude attenuation in signals

# reshape data to train model

x_test = map(abs, x_test)
x_train = map(abs, x_train)

test_samples, testx, testy, testz = np.shape(x_test)
train_samples, trainx, trainy, trainz = np.shape(x_train)

x_test = np.reshape(x_test, (test_samples, testx*testy*testz))
x_train = np.reshape(x_train, (train_samples, trainx*trainy*trainz))

print(np.shape(x_test))
print(np.shape(x_train))

(100, 18000)
(300, 18000)


In [31]:
y_allPred = []
y_allVal = []

clf = svm.SVC(gamma='auto', kernel='rbf', decision_function_shape='ovo')
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
y_allPred = np.append(y_allPred, y_pred)
y_allVal = np.append(y_allVal, y_test)
print (accuracy_score(y_test, y_pred))

0.77
